In [1]:
# imports
from importlib import reload

import os # a module to interact with the operating system
os.environ["ENDPOINT_URL"]="http://rook-ceph-rgw-nautiluss3.rook"
import numpy as np

import pandas as pd
import xarray
import h5py
#import healpy as hp
import time

import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats

from ulmo import io as ulmo_io
from ulmo import plotting
from ulmo.llc import io as llc_io
from ulmo.llc import plotting as llc_plotting

from ulmo.utils import image_utils
from ulmo.analysis import figures as ulmo_figs
from ulmo.plotting import plotting as ulmo_plotting
from ulmo.analysis import spatial_plots as sp

# Load table

In [2]:
llc_uniform= ulmo_io.load_main_table( 's3://llc/Tables/test_uniform_r0.5_test.feather' )
llc_uniform.head()

Read main table: s3://llc/Tables/test_uniform_r0.5_test.feather


,lat,lon,row,col,datetime,LLC_file,pp_file,pp_root,field_size,mean_temperature,...,T10,pp_idx,pp_type,LL,U_mean,V_mean,U_rms,V_rms,UV_mean,UV_rms
0,71.725098,-177.172684,12895,10607,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,-1.467546,...,-1.541000,647491,0,648.981934,-0.099179,-0.001986,0.059771,0.063255,0.141285,0.082805
1,71.747978,-175.380554,12895,10676,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,-0.492408,...,-1.540001,384768,0,-513.771851,-0.008606,-0.042140,0.041484,0.108360,0.046431,0.037870
2,71.758400,-173.559555,12895,10746,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,3.271719,...,2.540999,283882,0,345.448669,0.070156,-0.045817,0.022577,0.044823,0.099820,0.029986
3,71.755905,-171.737854,12895,10816,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,2.945127,...,2.229000,613023,0,169.709839,0.081717,-0.010355,0.024675,0.039160,0.115622,0.034708
4,71.740822,-169.943695,12895,10885,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_test_preproc.h5,llc_std,64,2.364319,...,1.891000,537161,0,349.981262,0.039346,0.019852,0.023803,0.031211,0.057416,0.030541


In [3]:
llc_uniform.iloc[0]

lat                                                         71.725098
lon                                                       -177.172684
row                                                             12895
col                                                             10607
datetime                                          2011-09-30 00:00:00
LLC_file            s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...
pp_file                  s3://llc/PreProc/LLC_uniform_test_preproc.h5
pp_root                                                       llc_std
field_size                                                         64
mean_temperature                                            -1.467546
Tmin                                                           -1.579
Tmax                                                           -0.619
T90                                                         -1.351001
T10                                                            -1.541
pp_idx              

# Upload a table to be run thru ulmo

In [4]:
llc_noise = llc_uniform.drop( columns = 'LL')

In [5]:
llc_noise.pp_file = 's3://llc/PreProc/LLC_uniform_viirs_test_preproc.h5'

In [6]:
ulmo_io.write_main_table(llc_noise, 's3://llc/Tables/LLC_uniform_viirs_noise.parquet')

Wrote Analysis Table: s3://llc/Tables/LLC_uniform_viirs_noise.parquet


In [7]:
llc_noise.pp_file = 's3://llc/PreProc/LLC_uniform_modis_test_preproc.h5'

In [8]:
ulmo_io.write_main_table(llc_noise, 's3://llc/Tables/LLC_uniform_modis_noise.parquet')

Wrote Analysis Table: s3://llc/Tables/LLC_uniform_modis_noise.parquet


# Load the table that was run & add the LL values

In [11]:
llc_viirs= ulmo_io.load_main_table('s3://llc/Tables/LLC_uniform_viirs_noise.parquet')

Read main table: s3://llc/Tables/LLC_uniform_viirs_noise.parquet


In [12]:
with ulmo_io.open( 's3://llc/Evaluations/LLC_uniform_viirs_test_log_prob.h5', 'rb') as f: 
    pp_hf = h5py.File(f, 'r')
    data = pp_hf['valid'][()]
    pp_hf.close()

In [14]:
order = llc_viirs.sort_values('pp_idx').index
order

Int64Index([ 52943, 488840, 150082, 113911, 735234, 201829, 448577, 520838,
            105698, 379685,
            ...
            334797, 121100, 294876, 130650, 322832, 124765, 637911, 611397,
            747596, 318725],
           dtype='int64', length=781662)

In [15]:
LL_values = pd.Series( data=data, index=order)
LL_values

52943      567.220703
488840    -305.529907
150082     545.560913
113911     292.379089
735234     445.446533
             ...     
124765    1053.924316
637911     228.411438
611397     152.105103
747596     156.784668
318725    1011.207153
Length: 781662, dtype: float32

In [16]:
llc_viirs['LL'] = LL_values
llc_viirs.head()

,lat,lon,row,col,datetime,LLC_file,pp_file,pp_root,field_size,mean_temperature,...,T10,pp_idx,pp_type,U_mean,V_mean,U_rms,V_rms,UV_mean,UV_rms,LL
0,71.725098,-177.172684,12895,10607,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_viirs_test_prepro...,llc_std,64,-1.467546,...,-1.541000,647491,0,-0.099179,-0.001986,0.059771,0.063255,0.141285,0.082805,642.516968
1,71.747978,-175.380554,12895,10676,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_viirs_test_prepro...,llc_std,64,-0.492408,...,-1.540001,384768,0,-0.008606,-0.042140,0.041484,0.108360,0.046431,0.037870,-508.767731
2,71.758400,-173.559555,12895,10746,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_viirs_test_prepro...,llc_std,64,3.271719,...,2.540999,283882,0,0.070156,-0.045817,0.022577,0.044823,0.099820,0.029986,345.613159
3,71.755905,-171.737854,12895,10816,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_viirs_test_prepro...,llc_std,64,2.945127,...,2.229000,613023,0,0.081717,-0.010355,0.024675,0.039160,0.115622,0.034708,169.615540
4,71.740822,-169.943695,12895,10885,2011-09-30,s3://llc/ThetaUVSalt/LLC4320_2011-09-30T00_00_...,s3://llc/PreProc/LLC_uniform_viirs_test_prepro...,llc_std,64,2.364319,...,1.891000,537161,0,0.039346,0.019852,0.023803,0.031211,0.057416,0.030541,343.068726


# Upload the finished version

In [17]:
ulmo_io.write_main_table(llc_viirs, 's3://llc/Tables/LLC_uniform_viirs_noise.parquet')

Wrote Analysis Table: s3://llc/Tables/LLC_uniform_viirs_noise.parquet
